In [209]:
import numpy as np
from copy import deepcopy
from math import ceil, floor

class Bqf:
    def __init__(self, a, b, c):
        self.terms = [["x^2", a, lambda x, y: x^2],
                      ["xy", b, lambda x, y: x * y],
                      ["y^2", c, lambda x, y: y]]
        
        self.update_vals(a,b,c)
        
        self.mat = [
            [self.a, self.b / 2],
            [self.b / 2, c]
        ]
        
        self.det  = self.determinant()
        self.disc = self.discriminant()
        
        if (self.disc == 0): self.type = "Perfect square"
            
        elif (self.disc < 0):
            if (self.a > 0): self.type = "Positive definite"
            else:            self.type = "Negative definite"
                
        else:                self.type = "Indefinite"
        
        self.transform = []
        self.reduced = self.reduce()

        
    def __call__(self, x, y):
        return sum([i[1] * i[2](x,y) for i in self.terms])
    
    def __repr__(self):
        out = ""
        for i in range(len(self.terms)):
            if self.terms[i][1] != 0:
                
                if i > 0:
                    if self.terms[i][1] > 0:
                        out += "+ "
                    else:
                        out += "- "
                elif (self.terms[i][1] < 0):
                    out += "-"
                        
                if abs(self.terms[i][1]) != 1:
                    out += str(abs(self.terms[i][1]))
                    
                out += self.terms[i][0] + " "

        return out[:-1]
    
    def __eq__(self, other):
        if self.det != other.det:
            return False
        M = other.mat @ np.linalg.inv(self.mat) 
        print(self.mat @ M)
        return np.allclose(other.mat, self.mat @ M)
    
    def determinant(self):
        return (self.mat[0][0] * self.mat[1][1]) - (self.mat[0][1] * self.mat[1][0])
    
    def discriminant(self):
        return self.determinant() * -4    
    
    def is_reduced(self):
        return -abs(self.a) < self.b <= abs(self.a) < abs(self.c) or\
                0 < self.b <= abs(self.a) == abs(self.c)
    
    def update_vals(self, a, b, c):
        self.a = a
        self.b = b
        self.c = c
        
        self.terms[0][1] = a
        self.terms[1][1] = b
        self.terms[2][1] = c
        
    def reduce(self):
        if (self.is_reduced()):
            print("already reduced")
            return self
        
        red = deepcopy(self)
        
        while (not red.is_reduced()):
            print(red.mat)
            
            if (abs(red.a) > abs(red.c)):
                red.mat = np.matmul(red.mat, [[0, 1],[-1, 0]])
                print([[0, -1],[1, 0]])
            else:
                m = 0
                # find transformation matrix
                if (red.a > 0):
                    m = floor((abs(red.a) - red.b)/(2*red.a))
                else:
                    m = ceil((abs(red.a) - red.b)/(2*red.a))

                red.mat = np.matmul(red.mat, [[1, m],[0, 1]])
                print([[1, m],[0, 1]])
            
            red.update_vals(red.mat[0][0], red.mat[0][1] * 2, red.mat[1][1])
            
            assert red.mat[0][1] * 2 == red.b
            
            if (abs(red.a) == abs(red.c) and red.b < 0):
                red.mat = [
                    [0, 1],
                    [-1,0]
                ]
                print("oop")
                break
            
            input()
        
        print(red.mat)
        print("is reduced")
        assert red.mat[0][1] * 2 == red.b
        print(red.mat[0][1])
        print(red)
        return red

In [210]:
a = Bqf(2, -15, 3)
print(a.reduced)

[[2, -7.5], [-7.5, 3]]
[[1, 4], [0, 1]]

[[  2.    0.5]
 [ -7.5 -27. ]]
is reduced
0.5
2.0x^2 + xy - 27.0y^2
2.0x^2 + xy - 27.0y^2


### 3.4

#### 1. For each of the following, determine whether the form is positive definite, negative definite, or indefinite.
(a) $x^2+y^2$

(b) $-(x^2)-(y^2)$

(c) $x^2 - (2y^2)$

(d) $10x^2 - (9xy) + (8y^2)$

(e) $x^2-(3xy)+y^2$

(f) $17x^2 - (26xy) + 10y^2$

In [117]:
bqfs_1 = [
    (1, 0, 1),
    (-1, 0, -1),
    (1, 0, -2),
    (10, -9, 8),
    (1, -3, 1),
    (17, -26, 10)
]

for i in range(len(bqfs_1)):
    bqf = Bqf(*bqfs_1[i])
    print("({}) {} is {}".format(chr(ord('`')+ i + 1), bqf, bqf.type))

(a) x^2 + y^2 is Positive definite
(b) -x^2 - y^2 is Negative definite
(c) x^2 - 2y^2 is Indefinite
[[10, -5], [-5, 8]]
(d) 10x^2 - 9xy + 8y^2 is Positive definite
[[1, -2], [-2, 1]]
(e) x^2 - 3xy + y^2 is Indefinite
[[17, -13], [-13, 10]]
(f) 17x^2 - 26xy + 10y^2 is Positive definite


#### 2. Prove that the quadratic form $x^2-(2xy)+y^2$ has discriminant 0. Determine the class of integers represented by this form.

In [45]:
bqf = Bqf(1, -2, 1)

assert bqf_2.det == 0
print("{}'s determinant is {}".format(bqf, int(bqf.det)))

print("? :(")

x^2 - 2xy + y^2's determinant is 0
? :(


#### 4. Use the binomial theorem to give a formula for positive integers $x_k$ and $y_k$ such that $(3+2\sqrt{2})^k=x_k+y_k\sqrt{2}$.

#### Show that $(3-2\sqrt{2})^k=x_k-y_k\sqrt{2}$.

#### Deduce that $x_k^2-(2y^2_k)=1$ for $k=1,2,3, \cdot\cdot\cdot$.

#### Show that $(x_k, y_k)=1$ for each $k$.

#### Show that $x_{k+1}=3x_k+4k_k$ and $y_{k+1}=2x_k+3y_k$ for $k=1,2,3, \cdot\cdot\cdot$.

#### Show that $\{x_k\}$ and $\{y_k\}$ are strictly increasing sequences.

#### Conclude that the number $1$ has infinitely many proper representations by the quadratic form $x^2-(2y^2)$.

### 3.5

#### 1. Find a reduced form that is equivalent to the form $7x^2+25xy+23y^2$.

#### 5. Show that $x^2+5y^2$ and $2x^2+2xy+3y^2$ are the only reduced quadratic forms of discriminant $-20$.

#### Show that the first of these forms does not represent $2$, but that the second one does.

#### Deduce that these forms are inequivalent, and hence that $H(-20)=2$. 

#### Show that an odd prime $p$ is represented by at least one of these forms if and only if $p\equiv 1, 3, 7, $ or $9 \pmod{20}$.